In [ ]:
# Import libraries

import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection._search import ParameterGrid
from itertools import product
from scipy.optimize import root_scalar
from scipy.optimize import fsolve
from scipy import stats
from scipy.stats import norm
from numpy.random import default_rng

In [ ]:
%matplotlib inline

In [ ]:
# Import whatif

from whatif import Model
from whatif import get_sim_results_df

## **1A: Base Model**

In [ ]:
# Base inputs

fixed_cost = 60000
var_cost = 90
selling_price = 120
spf_scale = 10000000
spf_exponent = -1.8

In [ ]:
# Create class that calculates revenue and total cost

class SingleProductSPF(Model):
    def __init__(self, fixed_cost, var_cost, selling_price, spf_scale, spf_exponent):
        self.fixed_cost = fixed_cost
        self.var_cost = var_cost
        self.selling_price = selling_price
        self.spf_scale = spf_scale
        self.spf_exponent = spf_exponent
        
    def demand(self):
        '''
        Compute demand
        '''
        return self.spf_scale * self.selling_price ** self.spf_exponent
    
    def revenue(self):
        '''
        Compute revenue
        '''
        return (self.spf_scale * self.selling_price ** self.spf_exponent) * self.selling_price

    def total_cost(self):
        '''
        Compute total cost
        '''
        return (self.spf_scale * self.selling_price ** self.spf_exponent) * self.var_cost + self.fixed_cost
    
    def profit(self):
        '''
        Compute profit
        '''
        profit = self.revenue() - self.total_cost()
        return profit

In [ ]:
# Create a SingleProductSPF object

model_1 = SingleProductSPF(fixed_cost, var_cost, selling_price, spf_scale, spf_exponent)

In [ ]:
# Testing model_1 output

print(model_1.demand())
print(model_1.revenue())
print(model_1.total_cost())
print(model_1.profit())

<div class="alert alert-block alert-success">
<b>YAY I got the same values as the homework check values. Moving along.<b>
</div>

## 1B: 1-Way Data Table

In [ ]:
def data_table(Model, scenario_inputs, outputs):

    # Clone the model using deepcopy
    model_clone = copy.deepcopy(Model)
    
    # Create parameter grid
    dt_param_grid = list(ParameterGrid(scenario_inputs))
    
    # Create the table as a list of dictionaries
    results = []

    # Loop over the scenarios
    for params in dt_param_grid:
        # Create a result dictionary based on a copy of the scenario inputs
        result = copy.copy(params)
        # Loop over the list of requested outputs
        for output in outputs:
            # Compute the output.
            out_val = getattr(model_clone, output)()
            # Add the output to the result dictionary
            result[output] = out_val
        
        # Append the result dictionary to the results list
        results.append(result)

    # Convert the results list (of dictionaries) to a pandas DataFrame and return it
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
# Create a dictionary of base input values

base_inputs = {'fixed_cost': 60000, 
               'var_cost': 90, 
               'selling_price': 120, 
               'spf_scale': 10000000, 
               'spf_exponent': -1.8}

In [ ]:
# Create a new model with inputs specified by base_inputs dictionary

model_2 = SingleProductSPF(**base_inputs)
print(model_2)
model_2.profit()

In [ ]:
# Specify input ranges for scenarios (dictionary)

# 1-way table

dt_param_ranges_1 = {'selling_price': np.arange(90, 191, 10)}

# Specify desired outputs (list)

outputs = ['demand','profit']

# Use data_table function to create 1-way data table

m2_dt1_df = Model.data_table(model_2, dt_param_ranges_1, outputs)
m2_dt1_df

In [ ]:
# Create line graph to show relationship between selling price and profit

fig, ax1 = plt.subplots(figsize = ( 10 , 6 ))

# Plot line graph

sns.lineplot(x = "selling_price", y = "profit", ci = None, data = m2_dt1_df)

# Set label for X axis

ax1.set_xlabel( "Selling Price (USD)" , size = 12 )

# Set label for y-axis

ax1.set_ylabel( "Profit (USD)" , size = 12 )

# Make X and Y currency

ax1.yaxis.set_major_formatter('${x:1,.0f}')
ax1.xaxis.set_major_formatter('${x:1,.0f}')

# Create chart title

ax1.set_title( "Relationship between Selling Price and Profit" , size = 14 )
  

In [ ]:
# Create color line graph to show relationship between selling price and profit

fig, ax2 = plt.subplots(figsize = ( 10 , 6 ))

# Plot line graph

sns.set_style("darkgrid")
sns.scatterplot(x="selling_price", y="profit", hue="demand", data=m2_dt1_df, palette="viridis")

# Set label for X axis

ax2.set_xlabel( "Selling Price (USD)" , size = 12 )

# Set label for y-axis

ax2.set_ylabel( "Profit (USD)" , size = 12 )

# Make X and Y currency

ax2.yaxis.set_major_formatter('${x:1,.0f}')
ax2.xaxis.set_major_formatter('${x:1,.0f}')

# Create chart title

ax2.set_title( "Relationship between Selling Price and Profit" , size = 14 )

<div class="alert alert-block alert-success">
<b>This is not a linear relationship. Demand is at its highest when the sell price is at its lowest ($90) but once you factor in variable and fixed costs, profit is negative (costs outweigh the revenue). Though demand decreaes as selling price increaes, because the selling price is so high, profit is actually positive because the units multiplied by the selling price (revenue) is still greater than the cost to produce.<b>
</div>

## 1C: Break Even

In [ ]:
base_inputs = {'fixed_cost': 60000, 
               'var_cost': 90, 
               'selling_price': 120, 
               'spf_scale': 10000000, 
               'spf_exponent': -1.8}

In [ ]:
def goal_seek(Model, obj_fn, target, by_changing, a, b, N=100, verbose=False):
    
    # Clone the model
    model_clone = copy.deepcopy(Model)
    
    setattr(model_clone, by_changing, a)
    f_a_0 = getattr(model_clone, obj_fn)()
    setattr(model_clone, by_changing, b)
    f_b_0 = getattr(model_clone, obj_fn)()
    
    if (f_a_0 - target) * (f_b_0 - target) >= 0:
        # print("Bisection method fails.")
        return None
    
    # Initialize the end points
    a_n = a
    b_n = b
    for n in range(1, N+1):
        # Compute the midpoint
        m_n = (a_n + b_n)/2
        
        # Function value at midpoint
        setattr(model_clone, by_changing, m_n)
        f_m_n = getattr(model_clone, obj_fn)()
        
        # Function value at a_n
        setattr(model_clone, by_changing, a_n)
        f_a_n = getattr(model_clone, obj_fn)()
        
        # Function value at b_n
        setattr(model_clone, by_changing, b_n)
        f_b_n = getattr(model_clone, obj_fn)()
        
        if verbose:
            print(f"n = {n}, a_n = {a_n}, b_n = {b_n}, m_n = {m_n}, width = {b_n - a_n}")

        # Figure out which half the root is in, or if we hit it exactly, or if the search failed
        if (f_a_n - target) * (f_m_n - target) < 0:
            a_n = a_n
            b_n = m_n
            if verbose:
                print("Root is in left half")
        elif (f_b_n - target) * (f_m_n - target) < 0:
            a_n = m_n
            b_n = b_n
            if verbose:
                print("Root is in right half")
        elif f_m_n == target:
            if verbose:
                print("Found exact solution.")
            return m_n
        else:
            if verbose:
                print("Bisection method fails.")
            return None
    
    # If we get here we hit iteration limit, return best solution found so far
    if verbose:
        print("Reached iteration limit")
    return (a_n + b_n)/2

In [ ]:
goal_seek(model_2, 'profit', 0, 'selling_price', 100, 200, N=100, verbose=True)

<div class="alert alert-block alert-success">
<b>The break even selling price is $126.43.<b>
</div>

## 1D: 2-Way Data Table

In [ ]:
# 2-way table

dt_param_ranges_2 = {'selling_price': np.arange(90, 191, 10),
                     'var_cost': np.arange(85, 111, 5)}

# Specify desired outputs (list)

outputs = ['demand','revenue','total_cost','profit']

# Use data_table function to create 1-way data table

m2_dt2_df = Model.data_table(model_2, dt_param_ranges_2, outputs)
m2_dt2_df

## 1E: Simulation

In [ ]:
rg = default_rng(4470)
rg.random() # Generate one just to see it work

In [ ]:
# Reset base inputs

fixed_cost = 60000
var_cost = 90
selling_price = 120
spf_scale = 10000000

In [ ]:
# Create new model for simulation

model_sim = SingleProductSPF(fixed_cost=fixed_cost,
                             var_cost=var_cost,
                             selling_price=selling_price,
                             spf_scale=spf_scale,
                             spf_exponent=spf_exponent)

In [ ]:
num_reps = 100
spf_exponent_sim = rg.uniform(-2.0, -1.6, num_reps)

random_inputs = {'spf_exponent': spf_exponent_sim}

In [ ]:
scenario_inputs = {'selling_price': np.arange(90, 191, 10)}
list(ParameterGrid(scenario_inputs))

In [ ]:
# Set output variable

sim_outputs = ['profit']

In [ ]:
def create_parameter_grid(scenario_inputs):

    param_grid = []

    keys, values = zip(*scenario_inputs.items())

    for v in product(*values):
        params = dict(zip(keys, v))
        param_grid.append(params)

    return param_grid

def simulate(Model, random_inputs, outputs, scenario_inputs=None, keep_random_inputs=False):

        # Clone the model
        model_clone = copy.deepcopy(Model)

        # Update clone with random_inputs
        model_clone.update(random_inputs)

        # Store raw simulation input values if desired
        if keep_random_inputs:
            scenario_base_vals = vars(model_clone)
        else:
            scenario_base_vals = vars(Model)

        # Initialize output counters and containers
        scenario_num = 0
        scenario_results = []

        # Check if multiple scenarios
        if scenario_inputs is not None:
            # Create parameter grid for scenario inputs
            sim_param_grid = list(create_parameter_grid(scenario_inputs))

            # Scenario loop
            for params in sim_param_grid:
                model_clone.update(params)
                # Initialize scenario related outputs
                result = {}
                scenario_vals = copy.copy(params)
                result['scenario_base_vals'] = scenario_base_vals
                result['scenario_num'] = scenario_num
                result['scenario_vals'] = scenario_vals
                raw_output = {}

                # Output measure loop
                for output_name in outputs:
                    output_array = getattr(model_clone, output_name)()
                    raw_output[output_name] = output_array

                # Gather results for this scenario
                result['output'] = raw_output
                scenario_results.append(result)
                scenario_num += 1

            return scenario_results

        else:
            # Similar logic to above, but only a single scenario
            results = []
            result = {}

            result['scenario_base_vals'] = scenario_base_vals
            result['scenario_num'] = scenario_num
            result['scenario_vals'] = {}

            raw_output = {}
            for output_name in outputs:
                output_array = getattr(model_clone, output_name)()
                raw_output[output_name] = output_array

            result['output'] = raw_output
            results.append(result)

            return results

In [ ]:
model_sim_results = simulate(model_sim, random_inputs, sim_outputs, scenario_inputs)

In [ ]:
for scenario in model_sim_results:
    print(scenario['scenario_num'], scenario['scenario_vals'], scenario['output']['profit'].mean())

In [ ]:
# Look at random scenario

model_sim_results[4]

In [ ]:
# Gather results to create data frame

def get_sim_results_df(results):
    
    dfs = []    
    for r in results:
        df = pd.DataFrame(r['output'])
        df['scenario_num'] = r['scenario_num']
        for key, val in r['scenario_vals'].items():
            df[key] = val
            
        dfs.append(df)

    results_df = pd.concat(dfs)
    
    return results_df

In [ ]:
# Create data frame

model_sim_results_df = get_sim_results_df(model_sim_results)

In [ ]:
# Show data frame

model_sim_results_df

In [ ]:
# Create histogram(s) of profit

profit_histo_g = sns.FacetGrid(model_sim_results_df, col='selling_price', sharey=True, col_wrap=3)
profit_histo_g = profit_histo_g.map(plt.hist, "profit")

In [ ]:
# Create box plot just for fun

sns.boxplot(x="selling_price", y="profit", data=model_sim_results_df);

In [ ]:
# Probability profit is negative

print(stats.percentileofscore(model_sim_results_df.profit, 0) / 100.0)

<div class="alert alert-block alert-success">
<b>55% chance profit is negative...no bueno, senor.<b>
</div>

## 2A: Extended Model with Capacity Constraints

In [ ]:
# New inputs for the extended model

fixed_cost = 60000
var_cost = 90
selling_price = 120
spf_scale = 10000000
spf_exponent = -1.8
ot_threshold = 1000
ot_var_cost = 110
lost_demand_threshold= 1200

In [ ]:
# Create class that calculates revenue and total cost

class SingleProductSPF_EXT(Model):
    def __init__(self, fixed_cost, var_cost, selling_price, spf_scale, spf_exponent, ot_threshold, ot_var_cost, lost_demand_threshold):
        self.fixed_cost = fixed_cost
        self.var_cost = var_cost
        self.selling_price = selling_price
        self.spf_scale = spf_scale
        self.spf_exponent = spf_exponent
        self.ot_threshold = ot_threshold
        self.ot_var_cost = ot_var_cost
        self.lost_demand_threshold = lost_demand_threshold
        
    def demand(self):
        '''
        Compute demand
        '''
        return self.spf_scale * self.selling_price ** self.spf_exponent
    
    def num_sold(self):
        '''
        Compute the number of units sold, canot be greater than 1200
        '''
        return min(self.demand(), self.lost_demand_threshold)
    
    def total_var_cost(self):
        '''
        Compute variable cost ($90 < 1000, $110 for 1000 <= 1200)
        '''
        return (self.var_cost * min(self.num_sold(), self.ot_threshold)) + (self.ot_var_cost * max(0, self.num_sold() - self.ot_threshold))
    
    def revenue(self):
        '''
        Compute revenue
        '''
        return self.num_sold() * self.selling_price

    def total_cost(self):
        '''
        Compute total cost
        '''
        return self.total_var_cost() + self.fixed_cost
    
    def profit(self):
        '''
        Compute profit
        '''
        return self.revenue() - self.total_cost()
    
    def __str__(self):
        """
        String representation of SingleProductSPF_EXT inputs
        """
        return str(vars(self))

In [ ]:
# Create a 1-way data table for the extended model

def data_table(Model, scenario_inputs, outputs):

    # Clone the model using deepcopy
    model_clone = copy.deepcopy(Model)
    
    # Create parameter grid
    dt_param_grid = list(ParameterGrid(scenario_inputs))
    
    # Create the table as a list of dictionaries
    results = []

    # Loop over the scenarios
    for params in dt_param_grid:
        # Create a result dictionary based on a copy of the scenario inputs
        result = copy.copy(params)
        # Loop over the list of requested outputs
        for output in outputs:
            # Compute the output.
            out_val = getattr(model_clone, output)()
            # Add the output to the result dictionary
            result[output] = out_val
        
        # Append the result dictionary to the results list
        results.append(result)

    # Convert the results list (of dictionaries) to a pandas DataFrame and return it
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
# Resetting base inputs for extended model

base_inputs_EXT = {'fixed_cost': 60000, 
               'var_cost': 90, 
               'selling_price': 120, 
               'spf_scale': 10000000, 
               'spf_exponent': -1.8, 
               'ot_threshold': 1000,
               'ot_var_cost': 110,
               'lost_demand_threshold': 1200}

In [ ]:
# Create a new model with inputs specified by base_inputs_EXT dictionary

model_3_ext = SingleProductSPF_EXT(**base_inputs_EXT)
print(model_3_ext)
model_3_ext.profit()

In [ ]:
# Specify input ranges for scenarios (dictionary)

# 1-way table

dt_param_ranges_3 = {'selling_price': np.arange(90, 191, 10)}

# Specify desired outputs (list)

outputs = ['demand','num_sold','profit']

# Use data_table function to create 1-way data table

m3_dt3_ext_df = Model.data_table(model_3_ext, dt_param_ranges_3, outputs)
m3_dt3_ext_df

In [ ]:
# Create line graph to show relationship between selling price and profit in the first model and the extended model

fig, axEXT = plt.subplots()

Model1_profit = m2_dt1_df.profit
Model1_sp = m2_dt1_df.selling_price
Model2_profit = m3_dt3_ext_df.profit
Model2_sp = m3_dt3_ext_df.selling_price

plt.plot(Model1_sp, Model1_profit, '-r', label="OG Model")
plt.plot(Model2_sp, Model2_profit, '-m', label="EXT Model")
leg = axEXT.legend()

# Set label for X axis

axEXT.set_xlabel( "Selling Price (USD)" , size = 12 )

# Set label for y-axis

axEXT.set_ylabel( "Profit (USD)" , size = 12 )

# Make X and Y currency

axEXT.yaxis.set_major_formatter('${x:1,.0f}')
axEXT.xaxis.set_major_formatter('${x:1,.0f}')

# Create chart title

axEXT.set_title( "Relationship between Selling Price and Profit: OG and EXT Models" , size = 14 )

In [ ]:
# Find break even selling price on the extended model

def goal_seek(Model, obj_fn, target, by_changing, a, b, N=100, verbose=False):
    
    # Clone the model
    model_clone = copy.deepcopy(Model)
    
    setattr(model_clone, by_changing, a)
    f_a_0 = getattr(model_clone, obj_fn)()
    setattr(model_clone, by_changing, b)
    f_b_0 = getattr(model_clone, obj_fn)()
    
    if (f_a_0 - target) * (f_b_0 - target) >= 0:
        # print("Bisection method fails.")
        return None
    
    # Initialize the end points
    a_n = a
    b_n = b
    for n in range(1, N+1):
        # Compute the midpoint
        m_n = (a_n + b_n)/2
        
        # Function value at midpoint
        setattr(model_clone, by_changing, m_n)
        f_m_n = getattr(model_clone, obj_fn)()
        
        # Function value at a_n
        setattr(model_clone, by_changing, a_n)
        f_a_n = getattr(model_clone, obj_fn)()
        
        # Function value at b_n
        setattr(model_clone, by_changing, b_n)
        f_b_n = getattr(model_clone, obj_fn)()
        
        if verbose:
            print(f"n = {n}, a_n = {a_n}, b_n = {b_n}, m_n = {m_n}, width = {b_n - a_n}")

        # Figure out which half the root is in, or if we hit it exactly, or if the search failed
        if (f_a_n - target) * (f_m_n - target) < 0:
            a_n = a_n
            b_n = m_n
            if verbose:
                print("Root is in left half")
        elif (f_b_n - target) * (f_m_n - target) < 0:
            a_n = m_n
            b_n = b_n
            if verbose:
                print("Root is in right half")
        elif f_m_n == target:
            if verbose:
                print("Found exact solution.")
            return m_n
        else:
            if verbose:
                print("Bisection method fails.")
            return None
    
    # If we get here we hit iteration limit, return best solution found so far
    if verbose:
        print("Reached iteration limit")
    return (a_n + b_n)/2

In [ ]:
goal_seek(model_3_ext, 'profit', 0, 'selling_price', 100, 200, N=100, verbose=True)

<div class="alert alert-block alert-success">
<b>Comparing the EXT model to the OG model, the EXT model does not even start making a profit until selling price is somewhere between $140 and $160. After running goal seek, the break even sell price is $143.33 which is about 17 dollars more than the break even sell price in the OG model. The shapes of the curves are also different (as you can see yourself, lol). It looks like the optimal situation for the EXT model is to never go above 1200 units. The "negative profit" in this situation more so reads to me as "missed opportunities." Had they been able to meet the demand, they would have been able to make what they're "losing."<b>
</div>

## 2B: Goal Seek Strangeness

In [ ]:
# Try running the line in the HW assignment

print(f"Break even selling price: {Model.goal_seek(model_3_ext, 'profit', 0, 'selling_price', 90, 500):.2f}")

<div class="alert alert-block alert-success">
<b>I re-ran my goal seek above with 90 and 500 for a and b and I did not get any output. I get output whenever b is less than or equal to 450...so what happens at a sell price of $450? Let's create another data table that goes higher than the sell price in our original data table.
</div>

In [ ]:
# Specify input ranges for scenario

# 1-way table

dt_param_ranges_4 = {'selling_price': np.arange(75, 551, 25)} 

# Specify desired outputs

outputs = ['demand','num_sold','profit']

# Use data_table function to create 1-way data table

m4_dt4_ext_df = Model.data_table(model_3_ext, dt_param_ranges_4, outputs)
m4_dt4_ext_df

<div class="alert alert-block alert-success">
<b>Ah-ha, profit goes back to being negative at a sell price of 475. Let's find the exact price profit is negative. This time I'll shorten up my data frame to somehwere between 450 and 475.<b>
</div>

In [ ]:
# Specify input ranges for scenario

# 1-way table

dt_param_ranges_5 = {'selling_price': np.arange(450, 476, 5)} 

# Specify desired outputs

outputs = ['demand','num_sold','profit']

# Use data_table function to create 1-way data table

m5_dt5_ext_df = Model.data_table(model_3_ext, dt_param_ranges_5, outputs)
m5_dt5_ext_df

<div class="alert alert-block alert-success">
<b>...one step deeper...now I know profit goes negative between 450 and 455...<b>
</div>

In [ ]:
# Specify input ranges for scenario

# 1-way table

dt_param_ranges_6 = {'selling_price': np.arange(450, 456, 1)} 

# Specify desired outputs

outputs = ['demand','num_sold','profit']

# Use data_table function to create 1-way data table

m6_dt6_ext_df = Model.data_table(model_3_ext, dt_param_ranges_6, outputs)
m6_dt6_ext_df

<div class="alert alert-block alert-success">
<b>So it looks like goal seek will work with a "b" value up until 454 and it will crash at 455; let's try it.<b>
</div>

In [ ]:
# Try running the line in the HW assignment at b = 455

print(f"Break even selling price: {Model.goal_seek(model_3_ext, 'profit', 0, 'selling_price', 90, 455):.2f}")

In [ ]:
# Try running the line in the HW assignment at b = 454

print(f"Break even selling price: {Model.goal_seek(model_3_ext, 'profit', 0, 'selling_price', 90, 454):.2f}")

<div class="alert alert-block alert-success">
<b> Ta-daaaaaa :) A sell price of 454 worked but 455 did not so my ultimate guess as to why we get the error message at anything +455 for b is because profit goes back to being negative.<b>
</div>